# Preparation

[View in Colaboratory](https://colab.research.google.com/github/stikbuf/Language_Modeling/blob/master/Keras_Character_Aware_Neural_Language_Models.ipynb)

## Configure the cloud environment


### Mount Google Drive

In [ ]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse


# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()


# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
# If you got a "Transport endpoint is not connected." error. Please run this line first to unmount the drive.
# See https://stackoverflow.com/questions/49588113/google-colab-script-throws-transport-endpoint-is-not-connected?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
!fusermount -u drive

# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive
a = !ls drive/
print('Files in Drive:', a)
assert a!=[], 'Drive should not be empty!'

In [ ]:
local_path='./drive/share_with_me/AI/Character-aware_LM/'
#local_path='./'
import sys
sys.path.append(local_path)
!ls './drive/share_with_me/AI/Character-aware_LM/'

In [ ]:
import tensorflow as tf
#assert tf.test.gpu_device_name() != '', "GPU not avaliable!"
tf.test.gpu_device_name()

## Load data (Penn Tree bank -- PTB)

In [1]:
local_path='./'

In [2]:
from __future__ import print_function
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" # Use single card. THIS LINE MUST BE RUN BEFORE TENSORFLOW IS IMPORTED
import pylab
%pylab inline
import matplotlib.pyplot as plt
import matplotlib  
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd
import random

from reader import ptb_raw_data, ptb_producer # by Google

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.5/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['pylab']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"
/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
train_data, valid_data, test_data, word_to_id = ptb_raw_data(local_path + 'data') # tokens
id_to_word = dict((v, k) for k, v in word_to_id.items())
voc_size = len(id_to_word)
voc_size

10000

In [4]:
print('Train data size: {0}, Valid data size: {1}, Test data size: {2}\n'.
      format(len(train_data), len(valid_data), len(test_data)))
print('train/val/test_data is a list, some elements in train_data is', train_data[:10])

Train data size: 929589, Valid data size: 73760, Test data size: 82430

train/val/test_data is a list, some elements in train_data is [9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983]


In [5]:
id_to_word[voc_size]='<SS>' # Add start word token '<SS>'
id_to_word[voc_size+1]='<EE>' # Add end word token '<EE>'
word_to_id = dict((v, k) for k, v in id_to_word.items())
voc_size = len(id_to_word)
voc_size

10002

In [6]:
word_id = pd.DataFrame.from_dict(word_to_id, orient='index').sort_values(by=0, ascending=True)
word_id.columns = ['id']
print(word_id.head())
print(word_id.tail())

       id
the     0
<unk>   1
<eos>   2
N       3
of      4
              id
ssangyong   9997
swapo       9998
wachter     9999
<SS>       10000
<EE>       10001


In [7]:
id_word = pd.DataFrame.from_dict(id_to_word, orient='index')
id_word.columns = ['word']
print(id_word.head())
print(id_word.tail())

    word
0    the
1  <unk>
2  <eos>
3      N
4     of
            word
9997   ssangyong
9998       swapo
9999     wachter
10000       <SS>
10001       <EE>


In [8]:
' '.join([id_to_word[id] for id in train_data[:50]])

'aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter <eos> pierre <unk> N years old will join the board as a nonexecutive director nov. N <eos> mr. <unk> is chairman of <unk> n.v. the dutch'

In [9]:
seq_len = 35

# RNN baseline

## data generator

In [ ]:
import random

from tensorflow.python.keras.utils import to_categorical 

def gen_word_word(batch_size=128, dataset='train'):
    assert dataset in ['train', 'valid', 'test'], 'Dataset must be train or valid or test.'
    
    dic = {'train':train_data, 'valid':valid_data, 'test':test_data}
    data = dic[dataset]
    
    while True:
        rnd_idxs = list(range(len(data)-seq_len-1))
        random.shuffle(rnd_idxs)
        cnt = 0
        while cnt < len(rnd_idxs) - batch_size :
            X = np.array([[word_to_id['<SS>']] + data[i:i+seq_len] + [word_to_id['<EE>']]
                          for i in rnd_idxs[cnt:cnt+batch_size]])
            Y = X[:,1:]
            X = X[:,:-1]
            Y = to_categorical(Y)
            #print(X.shape)
            cnt += batch_size
            yield X, Y

## Keras model

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import GRU, Dense, Embedding, InputLayer, Dropout
from tensorflow.python.keras.optimizers import RMSprop

Add dropout between layers, see [Recurrent Neural Network Regularization](https://arxiv.org/abs/1409.2329)

In [ ]:
model = Sequential()

embedding_size = 128


model.add(Embedding(input_dim=voc_size,
                    output_dim=embedding_size,
                    name='inputEmbedding'))
model.add(GRU(units=128, return_sequences=True))
model.add(Dropout(0.3)) 
model.add(GRU(units=64, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(voc_size, activation='softmax'))

If we denote $w_{1:T} = [w_1, w_2,...,w_T ]$ to be the sequence of words in thes, training involves minimizing
the negative log-likelihood ($NLL$)
$$NLL = - \sum_{t=1}^{T} \log Pr (w_t | w_{1:t-1})$$
i.e. the Crossentropy loss (with out averaging).  
As is standard in language modeling, we use perplexity(PPL) to evaluate the performance of our models. Perplexity of a model over a sequence $[w_1, w_2,...,w_T ]$ is given by
$$PPL = e^\frac{NLL}{T} = e^{ave (Crossentropy)}$$
where $NLL/Crossentropy$ is calculated over the test set.


In [ ]:
# perplexity
def PPL(y_true, y_pred):
    return tf.exp(tf.reduce_mean(tf.keras.backend.categorical_crossentropy(y_true, y_pred)))

def ACC(y_true, y_pred):
    ACC = tf.equal(tf.argmax(y_true, axis = 2), 
                   tf.argmax(y_pred, axis = 2))
    ACC = tf.cast(ACC, tf.float32)

    return tf.reduce_mean(ACC)

In [ ]:
optimizer = RMSprop(lr=1e-3)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[ACC, PPL])

In [ ]:
model.summary()

In [ ]:
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint
import os
if not os.path.exists(local_path + 'model/'):
    os.mkdir(local_path + 'model/')

path_model = local_path + 'model/model.keras'    
tensorboard = TensorBoard(log_dir='log')
checkpoint = ModelCheckpoint(filepath=path_model, verbose=1,
                             monitor='val_PPL',mode='min' ,save_best_only='True')
# path_model = local_path + 'model/model.keras'
# model.save(path_model)

callback_lists=[tensorboard,checkpoint]

In [ ]:
history = model.fit_generator(generator=gen_word_word(), 
                           steps_per_epoch=50, epochs=125,
                           callbacks=callback_lists,
                           validation_data=gen_word_word(dataset='valid'),
                           validation_steps=30)

In [ ]:
logs = pd.DataFrame(history.history)

In [ ]:
print(logs.columns)
pylab.rcParams['figure.figsize'] = (13, 8)
logs.loc[1:,['PPL','val_PPL']].plot() # start with 1 makes the figure prettier

In [ ]:
# path_model = local_path + 'model/model.keras'
# model.save(path_model)

In [ ]:
from tensorflow.python.keras.models import load_model

model_restore = load_model(path_model, custom_objects={'ACC':ACC,'PPL': PPL})

In [ ]:
history = model_restore.fit_generator(generator=gen_word_word(), 
                           steps_per_epoch=50, epochs=3,
                           callbacks=callback_lists,
                           validation_data=gen_word_word(dataset='valid'),
                           validation_steps=30)

In [ ]:
logs = pd.DataFrame(history.history)
print(logs.columns)
pylab.rcParams['figure.figsize'] = (13, 8)
logs.loc[:,['PPL','val_PPL']].plot()

In [ ]:
def predict_seq(model, preSeq=None, genLen=seq_len, power=1):
    """ Predict a sequence with length genLen.
        arg:
            model: Keras model used to predict.
            preSeq: list. The leading sequence.
            genLen: float or np.inf. If power is equal to np.inf, then an argmax will be used. 
            power: Probility power.
    """
    preSeq = [word_to_id['<SS>']] if preSeq == None else [word_to_id['<SS>']] + preSeq   
    pointer = len(preSeq) - 1
    
    for _ in range(genLen):
        inputSeq = np.array([preSeq])
        prob = model.predict(inputSeq)[0, pointer, :]
        if power==np.inf:
            pred = np.argmax(prob)
        else:
            prob = np.power(prob, power)
            prob = prob / np.sum(prob)
            pred = np.random.choice(range(voc_size), p=prob)
        preSeq.append(pred)
        pointer = pointer + 1

    return preSeq, ' '.join([id_to_word[id] for id in preSeq])

In [ ]:
_, seq = predict_seq(model, power=1)
seq

# Character aware model

[Character-Aware Neural Language Models -- arxiv-1508.06615 -- AAAI 2016](https://arxiv.org/abs/1508.06615)

[Ref: Github/jarfo/kchar](https://github.com/jarfo/kchar)


![model](https://github.com/stikbuf/Language_Modeling/blob/master/Character%20aware.png?raw=true)


## Load data

### convert to text

In [ ]:
train_data_text = [id_to_word[idx] for idx in train_data]
valid_data_text = [id_to_word[idx] for idx in valid_data]
test_data_text = [id_to_word[idx] for idx in test_data]
total_data_text = train_data_text + valid_data_text + test_data_text

maxWordLen = max([len(word) for word in total_data_text])
maxWordLen += 2 # Inclued Start and End character

ds = pd.Series([len(word) for word in total_data_text])
plt.figure(figsize=(10,5))
ds.plot.hist(bins=range(1, maxWordLen))
plt.title('word length distribution, max={0}, min={1}'.
          format(ds.max(), ds.min()))

### merge chars

In [ ]:
chars = []
for word in total_data_text:
    chars.extend(list(word))
    
ds = pd.Series(chars)
plt.figure(figsize=(15,10))   
matplotlib.rc('xtick', labelsize=20)
matplotlib.rc('ytick', labelsize=15)
ds.value_counts().plot.bar()
plt.title('character distribution, total {0} characters(without \'S\' and \'E\')'.format(len(set(chars))))
plt.show()

chars = list(set(chars + ['S'] + ['E'] + [' ']))
# 'S' for word leading char, 'E' for word ending char, space for padding
id_to_chars = dict(enumerate(chars))
chars_to_id = dict((v, k) for k,v in id_to_chars.items())
num_chars = len(chars)

print('number of chars:', num_chars, '\n')
print(chars, '\n')
print(id_to_chars, '\n')
print(chars_to_id, '\n')

In [ ]:
def word_to_charId(wordId):
    # Convert word to a string of word Ids
    wordString = 'S' + id_to_word[wordId].center(maxWordLen - 2) + 'E'
    return [chars_to_id[char] for char in wordString]

def wordSeq_charSeq(bWordSeq):
    batch, seqLen = bWordSeq.shape
    bWordSeq = bWordSeq.ravel()
    charSeq = np.array([word_to_charId(wordId) for wordId in bWordSeq])
    return charSeq.reshape(batch, seqLen, -1)

word_to_charId(word_to_id['the']) # 

In [ ]:
import random

from tensorflow.python.keras.utils import to_categorical 

def gen_char_word(batch_size=128, dataset='train'):
    assert dataset in ['train', 'valid', 'test'], 'Dataset must be train or valid or test.'
    
    dic = {'train':train_data, 'valid':valid_data, 'test':test_data}
    data = dic[dataset]
    
    while True:
        rnd_idxs = list(range(len(data)-seq_len-1))
        random.shuffle(rnd_idxs)
        cnt = 0
        while cnt < len(rnd_idxs) - batch_size :
            X = np.array([[word_to_id['<SS>']] + data[i:i+seq_len] + [word_to_id['<EE>']]
                          for i in rnd_idxs[cnt:cnt+batch_size]])
            Y = X[:,1:]
            X = X[:,:-1]
            Y = to_categorical(Y)
            X = wordSeq_charSeq(X) 
            #print(X.shape)
            cnt += batch_size
            yield X, Y

In [ ]:
print(next(gen_char_word(batch_size=1, dataset='train'))[1][0][2].shape)

## option

In [ ]:
class Option():
    def __init__(self):
        self.batch_size = 20
        self.seq_length = seq_len + 1
        self.max_word_l = maxWordLen # Include Start and End character
        self.char_vocab_size = num_chars
        self.char_vec_size = 15
        self.feature_maps = [50,100,150,200,200,200,200]
        self.kernels = [1,2,3,4,5,6,7]
        self.highway_layers = 2
        self.num_lstm_layers = 2
        self.rnn_size = 128
        self.word_vocab_size = voc_size
        self.dropout = 0.5
        self.learing_rate = 1e-5
        
opt = Option()

## CNN
![CNN part](https://github.com/stikbuf/Language_Modeling/blob/master/Character%20aware-CNN.png?raw=true)

In [ ]:
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Concatenate, Reshape

def CNN(seq_length, length, feature_maps, kernels, x):

    concat_input = []
    for feature_map, kernel in zip(feature_maps, kernels):
        reduced_l = length - kernel + 1
        conv = Conv2D(feature_map, (1, kernel), activation='tanh', data_format="channels_last")(x)
        maxp = MaxPooling2D((1, reduced_l), data_format="channels_last")(conv)
        concat_input.append(maxp)

    x = Concatenate()(concat_input)
    x = Reshape((seq_length, sum(feature_maps)))(x)
    return x

## Highway Network  
[Srivastava et al.](https://arxiv.org/abs/1505.00387)

Input vector is $\textbf{y}$, then layer output $\textbf{z}$ is
$$\textbf{z = t} \odot g(\textbf{W}_H\textbf{y}+\textbf{b}_H) + \textbf{(1 - t)} \odot \textbf{y}$$
where 
$$\textbf{t} = \sigma(\textbf{W}_T\textbf{y}+\textbf{b}_T)$$

$\textbf{t}$ is called the
*transform gate*, and $(\textbf{1}−\textbf{t})$ is called the *carry gate*. 
Similar to the memory cells in LSTM networks, highway layers allow for training of deep networks by adaptively carrying some dimensions of the input directly to the output. By construction the dimensions of $\textbf{y}$ and $\textbf{z}$ have to match, and hence $\textbf{W}_T$ and $\textbf{W}_H$ are square matrices. **Basically, a highway layer is a dense layer with residual connection modulated by an adaptive gate.**

A keras model is also a keras layer! So you can combine some keras layers to design your own layer. This is useful when combining with TimeDistributed wrapper. [See section 3 in this blog](https://keunwoochoi.wordpress.com/2016/11/18/for-beginners-writing-a-custom-keras-layer/)

In [ ]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Dense, Activation, Multiply, Add, Lambda, Input
from tensorflow.python.keras.initializers import Constant
from tensorflow.python.keras.models import Model

class LambdaWithShape(Lambda):
#     def __init__(self, function, **kwargs):
#         super(LambdaWithShape, self).__init__(function, **kwargs)
    def compute_output_shape(self, input_shape):
        return input_shape

def Highway(value, nLayers, activation='tanh', gateBias=-3):
    dim = K.int_shape(value)[-1]
    gateBiasInitalizer = Constant(gateBias)
    for i in range(nLayers):
        tGate = Dense(units=dim, bias_initializer=gateBiasInitalizer)(value)
        tGate = Activation('sigmoid')(tGate)
        #cGate = Lambda(lambda x: 1.0-x)(tGate) # WARNING:tensorflow:All custom layers should implement the `compute_output_shape`
        cGate = LambdaWithShape(lambda x: 1.0-x)(tGate) # I do not specify output_shape
        transformed = Dense(units=dim, bias_initializer=gateBiasInitalizer)(value)
        transformed = Activation(activation)(value)
        transformedGate = Multiply()([tGate, transformed])
        identityGate = Multiply()([cGate, value])
        value = Add()([transformedGate, identityGate])
    return value

inputs = Input((sum(opt.feature_maps),))
HighwayLayer = Model(inputs=inputs, outputs=Highway(inputs, nLayers=opt.highway_layers))

## Model

In [ ]:
from tensorflow.python.keras.layers import Input, Embedding, GRU, Dropout, BatchNormalization, TimeDistributed
#from tensorflow.python.keras.optimizers import SGD

#chars = Input(batch_shape=(opt.batch_size, opt.seq_length, opt.max_word_l), name='chars')
chars = Input(shape=(opt.seq_length, opt.max_word_l), name='chars') # will get a warning if you do not specify batch_shape
chars_embedding = Embedding(opt.char_vocab_size, opt.char_vec_size, name='chars_embedding')(chars)
cnn = CNN(opt.seq_length, opt.max_word_l, opt.feature_maps, opt.kernels, chars_embedding)
x = cnn
inputs = chars

x = BatchNormalization()(x)

x = TimeDistributed(HighwayLayer)(x)
highway = x

for l in range(opt.num_lstm_layers):
    #x = GRU(opt.rnn_size, return_sequences=True, stateful=True)(x)
    x = GRU(opt.rnn_size, return_sequences=True, stateful=False)(x)

    if opt.dropout > 0:
        x = Dropout(opt.dropout)(x)
        
output = Dense(opt.word_vocab_size, activation='softmax')(x)

modelCAware = Model(inputs=inputs, outputs=output)
modelCAware.summary()    

In [ ]:
modelConvWordFeatureBeforeHighway = Model(inputs=inputs, outputs=cnn)
modelConvWordFeatureAfterHighway = Model(inputs=inputs, outputs=highway)

In [ ]:
# perplexity
def PPL(y_true, y_pred):
    return tf.exp(tf.reduce_mean(tf.keras.backend.categorical_crossentropy(y_true, y_pred)))

def ACC(y_true, y_pred):
    ACC = tf.equal(tf.argmax(y_true, axis = 2), 
                   tf.argmax(y_pred, axis = 2))
    ACC = tf.cast(ACC, tf.float32)

    return tf.reduce_mean(ACC)

In [ ]:
from tensorflow.python.keras.optimizers import RMSprop

optimizer = RMSprop(lr=opt.learing_rate)
modelCAware.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=[ACC, PPL])  

#### Tensorboard histograms
Keras has a bug in TensorBoard visualization, see https://github.com/keras-team/keras/issues/3358  
DO NOT set show_hist_gram=True  unless you want TensorBoard visualization

In [ ]:
show_hist_gram = True 

In [ ]:
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint
import os
if not os.path.exists(local_path + 'model/'):
    os.mkdir(local_path + 'model/')

path_model = local_path + 'model/modelAware.keras'
if show_hist_gram:
    tensorboard = TensorBoard(log_dir='log', histogram_freq=1, write_grads=True)
else:
    tensorboard = TensorBoard(log_dir='log')
checkpoint = ModelCheckpoint(filepath=path_model, verbose=1,
                             monitor='val_PPL',mode='min' ,save_best_only='True')
# path_model = local_path + 'model/model.keras'
# model.save(path_model)

callback_lists=[tensorboard,checkpoint]

In [ ]:
if show_hist_gram:
    history = modelCAware.fit_generator(generator=gen_char_word(batch_size=opt.batch_size), 
                           steps_per_epoch=50, epochs=5,
                           callbacks=callback_lists, 
                           #validation_data=gen_char_word(batch_size=opt.batch_size, dataset='valid'),#ValueError: If printing histograms, validation_data must be provided, and cannot be a generator.
                           validation_data=next(gen_char_word(batch_size=(len(valid_data)-seq_len-2)//100, dataset='valid')),  # //100 to avoid memory error.        
                           validation_steps=None) 
else:
    history = modelCAware.fit_generator(generator=gen_char_word(batch_size=opt.batch_size), 
                           steps_per_epoch=50, epochs=5,
                           callbacks=callback_lists, 
                           validation_data=gen_char_word(batch_size=opt.batch_size, dataset='valid'),#ValueError: If printing histograms, validation_data must be provided, and cannot be a generator.
                           #validation_data=next(gen_char_word(batch_size=(len(valid_data)-seq_len-2)//100, dataset='valid')),  # //10 to avoid memory error. DO NOT use this line unless you want TensorBoard visualization       
                           validation_steps=30)

In [ ]:
logs = pd.DataFrame(history.history)
print(logs.columns)
pylab.rcParams['figure.figsize'] = (13, 8)
logs.loc[1:,['PPL','val_PPL']].plot()
plt.show()

## Word feature analysis
The convolution and the highway network can be viewed as a word feature extractor. Let's establish some intuition by the following experiments.

In [ ]:
def extractWordFeature(words, extcModel=modelConvWordFeatureAfterHighway):
    """
    words: List of word text.
    extcModel: Extractor model.
    
    returns: Pandas dataframe. Index is the word, corresponding to the feature vector.
    """
    
    padWord = lambda word: 'S' + word.lower().center(opt.max_word_l - 2) + 'E' # pad a single word
    wordsPadedCroped = [padWord(word[:opt.max_word_l-2]) for word in words] # cut input word if it is too long
    #print(wordsPadedCroped)
    wordsPadedCroped = [[chars_to_id[char] for char in word] for word in wordsPadedCroped] # convert to char index
    #print(wordsPadedCroped)
    
    # split into batches, list of tuples
    batches = []
    while wordsPadedCroped != []:
        if len(wordsPadedCroped)>opt.seq_length:
            batches.append((np.array(wordsPadedCroped[:opt.seq_length]), opt.seq_length))
            wordsPadedCroped = wordsPadedCroped[opt.seq_length:]
        else:
            wLast = wordsPadedCroped + [[chars_to_id[' ']]*opt.max_word_l]*(opt.seq_length-len(wordsPadedCroped))         
            batches.append((np.array(wLast), len(wordsPadedCroped)))
            wordsPadedCroped = []   
    
    features = []
    for batch in batches:
        data = batch[0]
        validNum = batch[1]
        features.append(extcModel.predict(np.expand_dims(data,0))[0,:validNum,:])
    
    features = np.vstack(features)
    
    # return a dataframe
    features = pd.DataFrame(data=features, index=words)
    features.index.name = 'featVecs'
    features.columns.name = 'vecDims'
    
    return features

In [ ]:
sampleWordList = ['look','looks','looked','looking','lok','looooooook',
                  'lk','loop','lock','locked','cook','see','observation',
                  'hear','run','reading','news','book','computer',
                  'programming','python','java','lisp','c#','matlab','jupyter']
vocWordList = list(word_id.index)
print(vocWordList[:5], '......', vocWordList[-4:])

In [ ]:
from scipy.spatial.distance import cdist

def sortedWordsByDistance(queryWord, Words, metric='cosine', 
                          extcModel=modelConvWordFeatureBeforeHighway):
    """
    queryWord: Single query word
    Words: Words to compare
    metric: metrics -- 'cosine',euclidean','correlation',...
    """
    queryWordFeat = extractWordFeature([queryWord], extcModel=extcModel)
    wordFeats = extractWordFeature(Words, extcModel=extcModel) 
    
    dis = pd.Series(cdist(queryWordFeat, wordFeats, metric=metric)[0])
    dis.index = Words
    dis = dis.sort_values(ascending=True)
    return dis
    
    
sortedDis = sortedWordsByDistance('look', sampleWordList)
sortedDis

In [ ]:
smpFeatures = extractWordFeature(sampleWordList, extcModel=modelConvWordFeatureAfterHighway)
smpFeatures

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def featureVis(feautres, usePCA=True):
    """
    features: A pandas dataframe. Index should be words, values should be word features.
    """
    wordList = feautres.index
    featTransed = feautres.values
    if usePCA:
        featTransed = PCA(n_components=len(feautres.columns)//7).fit_transform(featTransed)
    featTransed = TSNE(n_components=2).fit_transform(featTransed)
    featTransed = pd.DataFrame(featTransed, index=wordList)
    featTransed.index.name = 'featVecs'
    featTransed.columns.name = 'vecDims'

    pylab.rcParams['figure.figsize'] = (13, 8)
    axes = featTransed.plot.scatter(x=0, y=1)
    for txt in wordList:
        axes.annotate(txt, (featTransed.loc[txt,0],featTransed.loc[txt,1]))
        
featureVis(smpFeatures, usePCA=True)

In [ ]:
vocFeatures = extractWordFeature(vocWordList, extcModel=modelConvWordFeatureAfterHighway)
vocFeatures.head(1000)

In [ ]:
%%time
sample = vocFeatures.sample(200)
featureVis(sample, usePCA=True)

# Gated CNN model

[Language Modeling with Gated Convolutional Networks -- arxiv-1612.08083 -- Facebook AI Research](https://arxiv.org/abs/1612.08083)
![Gated CNN model](https://github.com/stikbuf/Language_Modeling/blob/master/Gated%20CNN.png?raw=true)

## Data generator

Same as RNN baseline.

In [28]:
import random

from tensorflow.python.keras.utils import to_categorical 

def gen_word_word(batch_size=128, dataset='train'):
    assert dataset in ['train', 'valid', 'test'], 'Dataset must be train or valid or test.'
    
    dic = {'train':train_data, 'valid':valid_data, 'test':test_data}
    data = dic[dataset]
    
    while True:
        rnd_idxs = list(range(len(data)-seq_len-1))
        random.shuffle(rnd_idxs)
        cnt = 0
        while cnt < len(rnd_idxs) - batch_size :
            X = np.array([[word_to_id['<SS>']] + data[i:i+seq_len] + [word_to_id['<EE>']]
                          for i in rnd_idxs[cnt:cnt+batch_size]])
            Y = X[:,1:]
            X = X[:,:-1]
            Y = to_categorical(Y)
            #print(X.shape)
            cnt += batch_size
            yield X, Y

## Model

Model
![model configuration](https://github.com/stikbuf/Language_Modeling/blob/dev/figures/gCNNConfig.png?raw=true)

I call:  
"(4, 1268)" a unit  
"[(4, 1268), (4, 1268)]" a block  
"[(4, 1268), (4, 1268)] * 12" a chunk  
A model is made by chunks.

In [29]:
from tensorflow.python.keras.layers import Conv1D, Multiply, Add, Input, Dense, Embedding
from tensorflow.python.keras.models import Model
from tensorflow.python.keras import backend as K

def gatedCNNUnit(kernel_size=3, filters=1024, input_shape=None):
    #print(input_shape)
    inputs = Input(shape=input_shape)
    conv = Conv1D(filters=filters, kernel_size=kernel_size,
                  strides=1, padding='same')(inputs)
    gated = Conv1D(filters=filters, kernel_size=kernel_size, 
                   strides=1, padding='same', activation='sigmoid')(inputs)
    value = Multiply()([conv, gated])
    print(K.int_shape(value)[1:])
    return Model(inputs=inputs, outputs=value)
    
    
def gatedCNNChunk(chunk, input_shape):
    origin = Input(shape=input_shape)
    inputs = origin
    for block in chunk:
        for unit in block:
            kernel_size, filters = unit
            x = gatedCNNUnit(kernel_size=kernel_size, filters=filters, input_shape=K.int_shape(inputs)[1:])(inputs)
        if K.int_shape(inputs)[-1] != K.int_shape(x)[-1]:
            inputs = Dense( K.int_shape(x)[-1])(inputs)
        x = Add()([inputs, x])
        inputs = x
    return Model(inputs=origin, outputs=x)

In [37]:
# embedding_size=128 in the paper
GCNN_13 = [ [(4, 1268)] ]*1 ,\
          [ [(4,1268), (4,1268)] ]*12 
    
GCNN_14B = [ [(5, 512)] ]*1 ,\
           [ [(1,128), (5,128), (1,512)] ]*3 ,\
           [ [(1,512), (5,512), (1,1024)] ]*3 ,\
           [ [(1,1024), (5,1024), (1,2048)] ]*6 ,\
            [ [(1,1024), (5,1024), (1,4096)] ]*1 
            
GCNN_9 = [ [(4, 807)] ]*1 ,\
          [ [(4,807), (4,807)] ]*4  
    
GCNN_8B = [ [(1, 512)] ]*1 ,\
           [ [(1,128), (5,128), (1,512)] ]*3 ,\
           [ [(1,256), (5,256), (1,512)] ]*3 ,\
           [ [(1,1024), (1,1024), (1,2048)] ]*1 

# embedding_size=208 in the paper
GCNN_8 = [ [(4, 900)] ]*1 ,\
           [ [(4,900)] ]*7

GCNN_14 = [ [(6, 850)] ]*3 ,\
           [ [(1,850)] ]*1 ,\
           [ [(5,850)] ]*4 ,\
           [ [(1,850)] ]*1 ,\
            [ [(4,850)] ]*3 ,\
            [ [(4,1024)] ]*1 ,\
            [ [(4,2048)] ]*1  
    
modelParam = GCNN_8
modelParam

([[(4, 900)]],
 [[(4, 900)],
  [(4, 900)],
  [(4, 900)],
  [(4, 900)],
  [(4, 900)],
  [(4, 900)],
  [(4, 900)]])

In [38]:
embedding_size = 128

inputs = Input(shape=(None,))
x = Embedding(input_dim=voc_size,
              output_dim=embedding_size)(inputs)
for chunk in modelParam:
    x = gatedCNNChunk(chunk, K.int_shape(x)[1:])(x)

x = Dense(voc_size, activation='softmax')(x)
gCNNModel = Model(inputs=inputs, outputs=x)
gCNNModel.summary()

(None, 900)
use dense
(None, 900)
(None, 900)
(None, 900)
(None, 900)
(None, 900)
(None, 900)
(None, 900)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_235 (InputLayer)       (None, None)              0         
_________________________________________________________________
embedding_7 (Embedding)      (None, None, 128)         1280256   
_________________________________________________________________
model_236 (Model)            (None, None, 900)         1039500   
_________________________________________________________________
model_244 (Model)            (None, None, 900)         45372600  
_________________________________________________________________
dense_28 (Dense)             (None, None, 10002)       9011802   
Total params: 56,704,158
Trainable params: 56,704,158
Non-trainable params: 0
_________________________________________________________________


In [27]:
# perplexity
def PPL(y_true, y_pred):
    return tf.exp(tf.reduce_mean(tf.keras.backend.categorical_crossentropy(y_true, y_pred)))

def ACC(y_true, y_pred):
    ACC = tf.equal(tf.argmax(y_true, axis = 2), 
                   tf.argmax(y_pred, axis = 2))
    ACC = tf.cast(ACC, tf.float32)

    return tf.reduce_mean(ACC)

In [23]:
from tensorflow.python.keras.optimizers import RMSprop

optimizer = RMSprop(lr=1e-6)
gCNNModel.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[ACC, PPL])

In [24]:
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint
import os
if not os.path.exists(local_path + 'model/'):
    os.mkdir(local_path + 'model/')

path_model = local_path + 'model/model.keras'    
tensorboard = TensorBoard(log_dir='log')
checkpoint = ModelCheckpoint(filepath=path_model, verbose=1,
                             monitor='val_PPL',mode='min' ,save_best_only='True')
# path_model = local_path + 'model/model.keras'
# model.save(path_model)

callback_lists=[tensorboard,checkpoint]

In [17]:
history = gCNNModel.fit_generator(generator=gen_word_word(batch_size=16), 
                           steps_per_epoch=50, epochs=125,
                           callbacks=callback_lists,
                           validation_data=gen_word_word(dataset='valid'),
                           validation_steps=30)

Epoch 1/125
49/50 [============================>.] - ETA: 0s - loss: 8.5283 - ACC: 0.0634 - PPL: 5718.8555
Epoch 00001: val_PPL improved from inf to 2463.87805, saving model to ./model/model.keras
50/50 [==============================] - 43s 861ms/step - loss: 8.5131 - ACC: 0.0647 - PPL: 5651.6442 - val_loss: 7.8089 - val_ACC: 0.0996 - val_PPL: 2463.8781
Epoch 2/125
49/50 [============================>.] - ETA: 0s - loss: 7.4296 - ACC: 0.1177 - PPL: 1729.1289
Epoch 00002: val_PPL improved from 2463.87805 to 1278.11442, saving model to ./model/model.keras
50/50 [==============================] - 43s 852ms/step - loss: 7.4280 - ACC: 0.1179 - PPL: 1725.6833 - val_loss: 7.1521 - val_ACC: 0.1433 - val_PPL: 1278.1144
Epoch 3/125
49/50 [============================>.] - ETA: 0s - loss: 7.0025 - ACC: 0.1612 - PPL: 1119.8093
Epoch 00003: val_PPL improved from 1278.11442 to 897.95807, saving model to ./model/model.keras
50/50 [==============================] - 45s 902ms/step - loss: 7.0046 - ACC

Epoch 24/125
49/50 [============================>.] - ETA: 0s - loss: 4.7359 - ACC: 0.4251 - PPL: 115.7530
Epoch 00024: val_PPL did not improve
50/50 [==============================] - 37s 740ms/step - loss: 4.7401 - ACC: 0.4247 - PPL: 116.2560 - val_loss: 4.7643 - val_ACC: 0.4225 - val_PPL: 117.4359
Epoch 25/125
49/50 [============================>.] - ETA: 0s - loss: 4.7548 - ACC: 0.4252 - PPL: 117.4271
Epoch 00025: val_PPL improved from 117.07078 to 111.23944, saving model to ./model/model.keras
50/50 [==============================] - 50s 997ms/step - loss: 4.7590 - ACC: 0.4245 - PPL: 117.9478 - val_loss: 4.7100 - val_ACC: 0.4265 - val_PPL: 111.2394
Epoch 26/125
49/50 [============================>.] - ETA: 0s - loss: 4.7138 - ACC: 0.4276 - PPL: 113.3617
Epoch 00026: val_PPL improved from 111.23944 to 104.42308, saving model to ./model/model.keras
50/50 [==============================] - 49s 982ms/step - loss: 4.7109 - ACC: 0.4280 - PPL: 113.0159 - val_loss: 4.6457 - val_ACC: 0.436

50/50 [==============================] - 50s 1s/step - loss: 3.9875 - ACC: 0.5211 - PPL: 55.0477 - val_loss: 3.9939 - val_ACC: 0.5246 - val_PPL: 54.3862
Epoch 48/125
49/50 [============================>.] - ETA: 0s - loss: 4.0244 - ACC: 0.5190 - PPL: 56.8463
Epoch 00048: val_PPL improved from 54.38625 to 52.48270, saving model to ./model/model.keras
50/50 [==============================] - 48s 962ms/step - loss: 4.0246 - ACC: 0.5191 - PPL: 56.8389 - val_loss: 3.9584 - val_ACC: 0.5302 - val_PPL: 52.4827
Epoch 49/125
49/50 [============================>.] - ETA: 0s - loss: 3.9886 - ACC: 0.5238 - PPL: 54.8876
Epoch 00049: val_PPL improved from 52.48270 to 51.39431, saving model to ./model/model.keras
50/50 [==============================] - 57s 1s/step - loss: 3.9881 - ACC: 0.5238 - PPL: 54.8424 - val_loss: 3.9381 - val_ACC: 0.5324 - val_PPL: 51.3943
Epoch 50/125
49/50 [============================>.] - ETA: 0s - loss: 3.9273 - ACC: 0.5293 - PPL: 51.8083
Epoch 00050: val_PPL improved from

49/50 [============================>.] - ETA: 0s - loss: 3.3954 - ACC: 0.5942 - PPL: 30.4969
Epoch 00071: val_PPL improved from 30.71492 to 29.98598, saving model to ./model/model.keras
50/50 [==============================] - 50s 1s/step - loss: 3.3953 - ACC: 0.5946 - PPL: 30.4809 - val_loss: 3.3984 - val_ACC: 0.5992 - val_PPL: 29.9860
Epoch 72/125
49/50 [============================>.] - ETA: 0s - loss: 3.3735 - ACC: 0.5939 - PPL: 29.6577
Epoch 00072: val_PPL improved from 29.98598 to 29.22564, saving model to ./model/model.keras
50/50 [==============================] - 47s 950ms/step - loss: 3.3707 - ACC: 0.5942 - PPL: 29.5726 - val_loss: 3.3723 - val_ACC: 0.6025 - val_PPL: 29.2256
Epoch 73/125
49/50 [============================>.] - ETA: 0s - loss: 3.3632 - ACC: 0.6001 - PPL: 29.5922
Epoch 00073: val_PPL improved from 29.22564 to 27.96350, saving model to ./model/model.keras
50/50 [==============================] - 49s 970ms/step - loss: 3.3648 - ACC: 0.6002 - PPL: 29.6244 - val_l

Epoch 95/125
49/50 [============================>.] - ETA: 0s - loss: 2.8701 - ACC: 0.6541 - PPL: 17.9744
Epoch 00095: val_PPL improved from 18.73024 to 18.20503, saving model to ./model/model.keras
50/50 [==============================] - 47s 948ms/step - loss: 2.8650 - ACC: 0.6543 - PPL: 17.8879 - val_loss: 2.8990 - val_ACC: 0.6584 - val_PPL: 18.2050
Epoch 96/125
49/50 [============================>.] - ETA: 0s - loss: 2.9446 - ACC: 0.6443 - PPL: 19.4890
Epoch 00096: val_PPL improved from 18.20503 to 18.08504, saving model to ./model/model.keras
50/50 [==============================] - 49s 971ms/step - loss: 2.9461 - ACC: 0.6443 - PPL: 19.5107 - val_loss: 2.8932 - val_ACC: 0.6597 - val_PPL: 18.0850
Epoch 97/125
49/50 [============================>.] - ETA: 0s - loss: 2.8663 - ACC: 0.6547 - PPL: 17.8291
Epoch 00097: val_PPL improved from 18.08504 to 17.56852, saving model to ./model/model.keras
50/50 [==============================] - 50s 993ms/step - loss: 2.8660 - ACC: 0.6545 - PPL:

Epoch 119/125
49/50 [============================>.] - ETA: 0s - loss: 2.4422 - ACC: 0.7048 - PPL: 11.7102
Epoch 00119: val_PPL improved from 12.54372 to 12.32998, saving model to ./model/model.keras
50/50 [==============================] - 48s 967ms/step - loss: 2.4399 - ACC: 0.7049 - PPL: 11.6814 - val_loss: 2.5094 - val_ACC: 0.7035 - val_PPL: 12.3300
Epoch 120/125
49/50 [============================>.] - ETA: 0s - loss: 2.3918 - ACC: 0.7065 - PPL: 11.1155
Epoch 00120: val_PPL improved from 12.32998 to 12.18490, saving model to ./model/model.keras
50/50 [==============================] - 49s 986ms/step - loss: 2.3889 - ACC: 0.7069 - PPL: 11.0826 - val_loss: 2.4976 - val_ACC: 0.7053 - val_PPL: 12.1849
Epoch 121/125
49/50 [============================>.] - ETA: 0s - loss: 2.4600 - ACC: 0.7012 - PPL: 11.9839
Epoch 00121: val_PPL improved from 12.18490 to 11.68638, saving model to ./model/model.keras
50/50 [==============================] - 47s 948ms/step - loss: 2.4612 - ACC: 0.7011 - P